Instalando módulos

In [ ]:
# ! pip install csv
# ! pip install pandas
# ! pip install chardet

Importanto Módulos

In [ ]:
import os
import csv
import pandas as pd
import chardet

In [ ]:
def listar_arquivos_originais(path_root_dir: str):
    
    list_dirs = os.listdir(path_root_dir)

    list_paths_dirs = []
    list_path_files = []

    for ano_dir in list_dirs:

        list_paths_dirs.append(os.path.join(path_root_dir, ano_dir))

    for path_dir in list_paths_dirs:

        list_files = os.listdir(path_dir)

        for file in list_files:

            list_path_files.append(os.path.join(path_dir, file))
    
    return list_path_files

In [ ]:
def identificar_encoding(path_file: str):
    with open(path_file, 'rb') as file:
        raw_data = file.read()

    result = chardet.detect(raw_data)
    encoding = result['encoding']
    
    return encoding

In [ ]:
def obter_dados_geograficos_estacao(path_file: str):

    with open(path_file, mode='r') as file:
        
        csv_reader = csv.reader(file)
        lines = list(csv_reader)

    lines = lines[:8]
    
    try:
        dados_estacao = {
        'regiao' : lines[0][0].split(';')[1],
        'uf' : lines[1][0].split(';')[1],
        'estacao' : lines[2][0].split(';')[1],
        'latitude' : lines[4][0].split(';')[1]+'.'+lines[4][1],
        'longitude' : lines[5][0].split(';')[1]+'.'+lines[5][1],
        'altitude' : lines[6][0].split(';')[1],
        }
        return dados_estacao
    
    except Exception as e:
     
        import inspect
        print(f"{inspect.getframeinfo(inspect.currentframe()).function};{e};{path_file}")

    finally:

        pass


In [ ]:
def separar_dados_meteorologicos(path_file: str):

    path_dir_destino_dados_meteorologicos = r'..\data\dados_tratados\dados_meteorologicos_separados'
    path_arquivo_dados_meteorologicos =  os.path.join(path_dir_destino_dados_meteorologicos, path_file.split("\\")[-1])

    # Abrindo o arquivo original e o novo arquivo em modo de escrita
    with open(path_file, 'r') as file, open(path_arquivo_dados_meteorologicos, 'w', newline='') as path_arquivo_dados_meteorologicos:

        # Cria um objeto csv.reader para ler o arquivo original
        reader = csv.reader(file)
        # Cria um objeto csv.writer para escrever no arquivo de destino
        writer = csv.writer(path_arquivo_dados_meteorologicos)

        # Ignora as primeiras 8 linhas
        for _ in range(8):
            next(reader)

        # Copia as linhas restantes para o arquivo de destino
        for row in reader:
            writer.writerow(row)

In [ ]:
def obter_dados_meteorologicos(path_arquivo_dados_meteorologicos: str, encoding: str):

    df = pd.read_csv(path_arquivo_dados_meteorologicos, sep=';', encoding=encoding)
    
    try:

        df['data_hora'] = df['DATA (YYYY-MM-DD)'] + ' ' + df['HORA (UTC)']
        df['data_hora'] = pd.to_datetime(df['data_hora'], format='%Y-%m-%d %H:%M')
        df.drop('Unnamed: 19', axis=1, inplace=True)
        df.drop('DATA (YYYY-MM-DD)', axis=1, inplace=True)
        df.drop('HORA (UTC)', axis=1, inplace=True)
        return df

    except Exception as e:
        
        import inspect
        print(f"{inspect.getframeinfo(inspect.currentframe()).function};{e};{path_arquivo_dados_meteorologicos}")

    finally:

        pass


In [ ]:
def integrar_dados_geograficos_meteorologicos(dados_meteorologicos: pd.DataFrame, dados_geograficos: dict):

    df = dados_meteorologicos
    
    try:
    
        for chave_valor in dados_geograficos.items():

            dados_meteorologicos[f'{chave_valor[0]}'] = f'{chave_valor[1]}'
            
        # renomeando as colunas do dataframe
        df.columns = [
            'precip_total_horario[mm]',
            'pres_atm_estacao_horaria[mB]',
            'pres_atm_max_hora_ant[mB]',
            'pres_atm_min_hora_ant[mB]',
            'radiacao_global[KJ/m2]',
            'temp_ar_bulbo_seco_horario[oC]',
            'temp_ponto_orvalho[oC]',
            'temp_max_hora_ant[oC]',
            'temp_min_hora_ant[oC]',
            'temp_orv_max_hora_ant[oC]',
            'temp_orv_min_hora_ant[oC]',
            'umid_relat_ar_max_hora_ant[%]',
            'umid_relat_ar_min_hora_ant[%]',
            'umid_relat_ar_hora_atual[%]',
            'vento_direcao_hora[gr]',
            'vento_rajada_max_hora[m/s]',
            'vento_vel_hora[m/s]',
            'data_hora',
            'regiao', 
            'uf',
            'estacao_cidade', 
            'estacao_latitude', 
            'estacao_longitude', 
            'estacao_altitude',
            ]
        
        # reordenando as colunas do dataframe
        df = df.reindex(columns=[
            'data_hora',
            'estacao_cidade', 
            'uf',
            'regiao', 
            'estacao_latitude', 
            'estacao_longitude', 
            'estacao_altitude',
            'precip_total_horario[mm]',
            'pres_atm_estacao_horaria[mB]',
            'pres_atm_max_hora_ant[mB]',
            'pres_atm_min_hora_ant[mB]',
            'radiacao_global[KJ/m2]',
            'temp_ar_bulbo_seco_horario[oC]',
            'temp_ponto_orvalho[oC]',
            'temp_max_hora_ant[oC]',
            'temp_min_hora_ant[oC]',
            'temp_orv_max_hora_ant[oC]',
            'temp_orv_min_hora_ant[oC]',
            'umid_relat_ar_max_hora_ant[%]',
            'umid_relat_ar_min_hora_ant[%]',
            'umid_relat_ar_hora_atual[%]',
            'vento_direcao_hora[gr]',
            'vento_rajada_max_hora[m/s]',
            'vento_vel_hora[m/s]',
            ])
    
        return df
            
    except Exception as e:
        
        import inspect
        print(f"{inspect.getframeinfo(inspect.currentframe()).function};{e};{None}")

    finally:
        
        pass


In [ ]:
path_dados_meteorologicos_dir = r'..\data\dados-meteorologicos'
path_dados_meteorologicos_separados_dir = r"..\data\dados_tratados\dados_meteorologicos_separados"

list_path_files = listar_arquivos_originais(path_dados_meteorologicos_dir)

df_dados_meteorologicos = pd.DataFrame(columns=[
        'data_hora',
        'estacao_cidade', 
        'uf',
        'regiao', 
        'estacao_latitude', 
        'estacao_longitude', 
        'estacao_altitude',
        'precip_total_horario[mm]',
        'pres_atm_estacao_horaria[mB]',
        'pres_atm_max_hora_ant[mB]',
        'pres_atm_min_hora_ant[mB]',
        'radiacao_global[KJ/m2]',
        'temp_ar_bulbo_seco_horario[oC]',
        'temp_ponto_orvalho[oC]',
        'temp_max_hora_ant[oC]',
        'temp_min_hora_ant[oC]',
        'temp_orv_max_hora_ant[oC]',
        'temp_orv_min_hora_ant[oC]',
        'umid_relat_ar_max_hora_ant[%]',
        'umid_relat_ar_min_hora_ant[%]',
        'umid_relat_ar_hora_atual[%]',
        'vento_direcao_hora[gr]',
        'vento_rajada_max_hora[m/s]',
        'vento_vel_hora[m/s]',
        ])

for path_dados_meteorologicos_file in list_path_files:

    encoding = identificar_encoding(path_dados_meteorologicos_file)
    # separar_dados_meteorologicos(path_dados_meteorologicos_file)
    dados_geograficos = obter_dados_geograficos_estacao(path_dados_meteorologicos_file)
    
    path_dados_meteorologicos_separados_file = os.path.join(path_dados_meteorologicos_separados_dir, path_dados_meteorologicos_file.split('\\')[-1])
    dados_meteorologicos = obter_dados_meteorologicos(path_dados_meteorologicos_separados_file, encoding)
    df_file = integrar_dados_geograficos_meteorologicos(dados_meteorologicos, dados_geograficos)
    df_dados_meteorologicos = pd.concat([df_dados_meteorologicos, df_file], ignore_index=True)

df_dados_meteorologicos

In [ ]:
df_dados_meteorologicos.to_csv(r"..\data\df_dados_meteorologicos.csv", index=False, encoding='utf-8')